In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%cd ..
!ls
from core import *

/Users/trippgordon/Desktop/Code/sonic/NoisyGAN
Data        README.md   core        models      submit.sh
Notebooks   __init__.py depricated  notes_ideas train.py


INFO:absl:tf versionn: 2.1.0


In [57]:
class binary_crossentropy(tf.keras.layers.Layer):
    def __init__(self,name = "binary_crossentropy"):
        super().__init__(name=name)
        
    def call(self, target, output):
        return tf.keras.losses.binary_crossentropy(target,output)

In [136]:
class Discriminator(ddsp.training.models.Model):
    """for now, just re-encode the fake sample. 
        in the future, just feed flz from fake right in
        TODO: subclass ddsp Model
    """
    def __init__(self, losses=[binary_crossentropy()]):
        super().__init__(name='discriminator', losses=losses)
        self.preprocessor = self.buildPreprocessor()
        self.flzEncoder = self.buildFLZEncoder()
        self.classifier = self.buildClassifier()
        
    def call(self, sample, label=1, training=True): 
        # audio key must have shape (1,64000)
        preprocessed = self.preprocessor(sample)
        
        # only get flz if label=1? ... try both
        encoded = self.flzEncoder(preprocessed)
        
        # shape the shit into [1, 1000, 8, 1]
        encoded_concat = tf.concat([encoded['f0_scaled'],
                                    encoded['ld_scaled'],
                                    encoded['z']], axis=2)
        encoded_concat = tf.expand_dims(encoded_concat,axis=3)
        
        # classify if it's real or not
        classification = self.classifier(encoded_concat, training=training)
        
        if training:
            self.add_losses(label, classification)
            
        return classification
        
    def buildPreprocessor(self):
        return ddsp.training.preprocessing.DefaultPreprocessor(time_steps=1000)

    def buildFLZEncoder(self):
        # TODO: try giving this an f0 encoder, like in ae_abs.gin
        encoder = ddsp.training.encoders.MfccTimeDistributedRnnEncoder(z_dims=6,
                                                                       z_time_steps=1000)
        return encoder
    
    def buildClassifier(self,training=True):
        # now encode even further down to a binary classification real or fake
        discriminator = Sequential()
        # downsample to 500x3
        discriminator.add(Conv2D(16, (3,3), strides=(2, 2), padding='same', input_shape=(1000,8,1)))
        discriminator.add(BatchNormalization())
        discriminator.add(LeakyReLU(alpha=0.2))
        # downsample to 250 x 2
        discriminator.add(Conv2D(16, (3,3), strides=(2, 2), padding='same'))
        discriminator.add(BatchNormalization())
        discriminator.add(LeakyReLU(alpha=0.2))
        # downsample to 125 x 1
        discriminator.add(Conv2D(16, (3,3), strides=(2, 2), padding='same'))
        discriminator.add(BatchNormalization())
        discriminator.add(LeakyReLU(alpha=0.2))
        # classify
        discriminator.add(Flatten())
        discriminator.add(Dense(1, activation='sigmoid'))
#         discriminator.summary()
        
#         if training:
#             discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return discriminator

In [152]:
def getRealSample(sampleNum):
    # get a dataset
    output_tfrecord_path = './Data/piano/piano30s.tfrecord'
    dataset = DDSP_DATASET(output_tfrecord_path)
    # get a sample
    real_sample = dataset.getSample(sampleNum=sampleNum)
    for key in real_sample.keys():
        real_sample[key] = tf.expand_dims(real_sample[key],axis=[0]) 
    return real_sample

real_sample = getRealSample(0)
describeSample(real_sample)

X = tf.convert_to_tensor(real_sample)
y = tf.convert_to_tensor(1)

audio: (1, 64000) Range: -0.24915089 0.28982782 type: <class 'tensorflow.python.framework.ops.EagerTensor'>
f0_confidence: (1, 1000) Range: 0.019729322 0.9687585 type: <class 'tensorflow.python.framework.ops.EagerTensor'>
f0_hz: (1, 1000) Range: 41.480316 783.93646 type: <class 'tensorflow.python.framework.ops.EagerTensor'>
loudness_db: (1, 1000) Range: -84.711914 -58.525723 type: <class 'tensorflow.python.framework.ops.EagerTensor'>


ValueError: Attempt to convert a value ({'audio': <tf.Tensor: shape=(1, 64000), dtype=float32, numpy=
array([[-0.03097209, -0.03147123, -0.03178149, ..., -0.00474663,
         0.0056245 ,  0.01854765]], dtype=float32)>, 'f0_confidence': <tf.Tensor: shape=(1, 1000), dtype=float32, numpy=
array([[0.3072656 , 0.33819452, 0.27169603, 0.2694023 , 0.22989208,
        0.41040137, 0.23871318, 0.2181089 , 0.2328403 , 0.35295233,
        0.46525562, 0.18296625, 0.38165295, 0.21442135, 0.16370024,
        0.27864522, 0.28149882, 0.28636524, 0.2098315 , 0.20818925,
        0.31763706, 0.24693741, 0.15935491, 0.14993511, 0.21942833,
        0.4123883 , 0.25723958, 0.16390848, 0.2548882 , 0.17244558,
        0.14870746, 0.12055663, 0.19223295, 0.33223343, 0.44571662,
        0.22551419, 0.35050482, 0.17865764, 0.153492  , 0.19161339,
        0.23142087, 0.1056214 , 0.20979959, 0.21740426, 0.18540354,
        0.30512336, 0.19738461, 0.11859544, 0.17305262, 0.2193046 ,
        0.19075036, 0.1670129 , 

In [153]:
discriminator = Discriminator()

discriminator.compile(optimizer="adam")
d_loss = discriminator.train_on_batch(X,y)

TypeError: in converted code:

    <ipython-input-92-9aadac816885>:17 call  *
        encoded = self.flzEncoder(preprocessed)
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer.py:773 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/ddsp/training/encoders.py:51 call  *
        z = self.compute_z(conditioning)
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/ddsp/training/encoders.py:130 compute_z  *
        z = self.z_norm(mfccs[:, :, tf.newaxis, :])[:, :, 0, :]
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer.py:773 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/ddsp/training/nn.py:57 call  *
        x = normalize_op(x, self.norm_type)
    /Users/tripp

In [ ]:
# predict
discriminator = Discriminator()
discriminator(real_sample,label=1,training=False)